In [8]:
#Imports
import numpy as np
import sklearn as sk
import matplotlib.pyplot as plt


In [9]:
data_train = np.load('fashion_train.npy').astype(np.int8)
data_test = np.load('fashion_test.npy').astype(np.int8)


In [10]:
#Decision tree

#Load data

#Find split

#   For every pair of point and feature, find a split with lowest impurity

#   Store the split somewhere -- we don't have to store the

#   New split, consider all the points in the sub-trees. Again, choose the split with the lowest impurity measure.

#Choose a stopping condition: minimum information gain, maximum tree height, etc?

#Store the set of decision rules somehow - we don't have to store it in the initial tree.
#The tree can be built first and then the desicion rules can be written based on the thresholds and the class majority.

#OR Pruning?

#Interface for training and classifying?

In [11]:
class tree:
    
    class node:
        def __init__(self,  left = None, right = None, feature = None, threshold = None, prediction = None):
            self.left = left
            self.right = right
            self.feature = feature
            self.threshold = threshold
            self.prediction = prediction


    def __init__(self, root= None):
        self.root = root

    @staticmethod
    def gini_impurity(label_counts, total_count):
        if total_count == 0:
            return 0
        label_counts = np.array(label_counts)
        proportions = label_counts / total_count
        return 1 - np.sum(proportions ** 2)
    
    @staticmethod
    def binary_entropy(label_counts, total_count):
        if total_count == 0:
            return 0
        label_counts = np.array(label_counts)
        proportions = label_counts / total_count
        # Avoid log(0) by masking zero proportions
        valid_proportions = proportions[proportions > 0]
        return - np.sum(valid_proportions * np.log(valid_proportions))

    @staticmethod
    def split_single_variable_optimized(data, impur = "gini"):
        n = len(data)
        
        # Sort data by the feature (first column)
        sorted_data = data[data[:, 0].argsort()]
        
        # Total bincount for the whole dataset (right side initially) (Implemented as an array. Index = label_id)
        total_right = np.bincount(sorted_data[:, 1].astype(int))
        total_left = np.zeros_like(total_right)
        
        best_split_index = -1
        best_impurity = float('inf')

        # Cumulative sums to track left and right distributions
        for i in range(1, n):  # Iterate over sorted data (ignoring first since it's trivial)
            label = int(sorted_data[i - 1, 1])
            total_left[label] += 1  # Add this label to the left side
            total_right[label] -= 1  # Remove this label from the right side
            
            l_count = i  # Left side count
            r_count = n - i  # Right side count
            
            # Skip invalid splits where all data is on one side
            if l_count == 0 or r_count == 0:
                continue
            
            # Only evaluate splits when the feature value changes
            if sorted_data[i - 1, 0] != sorted_data[i, 0]:
                
                if impur == "gini":
                    impurity_l = tree.gini_impurity(total_left, l_count)
                    impurity_r = tree.gini_impurity(total_right, r_count)
                
                if impur == "entropy":
                    impurity_l = tree.binary_entropy(total_left, l_count)
                    impurity_r = tree.binary_entropy(total_right, r_count)
                    
                
                # Weighted average of left and right impurities
                weighted_impurity = (l_count / n * impurity_l) + (r_count / n * impurity_r)
                
                # Track the best split
                if weighted_impurity < best_impurity:
                    best_impurity = weighted_impurity
                    best_split_index = i
        
        # Return the feature value that defines the best split
        if best_split_index != -1:
            split_value = (sorted_data[best_split_index - 1, 0] + sorted_data[best_split_index, 0]) / 2
            return split_value, best_impurity
        else:
            return None, None  # No valid split found

    # Find the best split across all variables (features)
    @staticmethod
    def find_best_split(X, y, impur = "gini"):
        best_feature = None
        best_value = None
        best_impurity = float('inf')

        # Combine X and y into one array for convenience
        data = np.hstack((X, y[:, np.newaxis]))

        # Iterate over all features (columns in X)
        for feature_idx in range(X.shape[1]):
            feature_data = data[:, [feature_idx, -1]]  # Select the feature and the labels
            split_value, impurity = tree.split_single_variable_optimized(feature_data, impur)
            
            # Track the best split across all features
            if impurity is not None and impurity < best_impurity:
                best_impurity = impurity
                best_feature = feature_idx
                best_value = split_value
        
        return best_feature, best_value, best_impurity


    def fit(self, X, y, max_depth=7, depth = 0, impur = "gini"): #stole the depth tracker from Guilia hehe

        
        split = self.find_best_split(X, y, impur="gini")                
        if depth == max_depth or len(set(y)) == 1 or split[0] is None:
            p_class = np.bincount(y.astype(int)).argmax()
            return self.node(prediction=p_class)

        

        if split[0] != None:
            node = self.node(threshold = split[1], feature = split[0])
    
            Xy = np.hstack((X, y[:, np.newaxis]))
            Xy_left = Xy[(Xy[:, node.feature] <= node.threshold)]
            Xy_right = Xy[(Xy[:, node.feature] > node.threshold)]

            X_left = Xy_left[:, :-1]
            y_left = Xy_left[:, -1]

            X_right = Xy_right[:, :-1]
            y_right = Xy_right[:, -1]

            if self.root is None:
                self.root = node 
            
            if X_left.size > 0 and y_left.size > 0:
                node.left = self.fit(X_left, y_left, max_depth, depth=depth+1)
            else:
                p_class = np.bincount(y_right.astype(int)).argmax()
                return self.node(prediction=p_class)
            if X_right.size > 0 and y_right.size > 0:
                node.right = self.fit(X_right, y_right, max_depth, depth=depth+1)
            else:
                p_class = np.bincount(y_left.astype(int)).argmax()
                return self.node(prediction=p_class)
              
        return node
        

    def singlepredict(self, X, root=None):
        if root is None:
            root = self.root

        if root.prediction is not None:
            return root.prediction
        
        if X[root.feature] < root.threshold:
            return self.singlepredict(X, root.left)
        else:
            return self.singlepredict(X, root.right)



In [12]:
X = data_train[:, :-1]
y = data_train[:, -1]

In [13]:
print(X)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [14]:
TREE = tree()
TREE.fit(X, y, 7, impur = "entropy")

C:\Users\pauls\AppData\Local\Temp\ipykernel_23716\1944499316.py:82: RuntimeWarning: overflow encountered in scalar add
  split_value = (sorted_data[best_split_index - 1, 0] + sorted_data[best_split_index, 0]) / 2


In [19]:
predictions = []

for i in range(int(len(data_test))):
    xtest = data_test[i, :-1]
    predictions.append(TREE.singlepredict(xtest))


   

In [20]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(data_test[:,-1], predictions)
accuracy, np.unique(predictions)

(0.7518, array([0, 1, 2, 3, 4], dtype=int64))